In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import levene, shapiro, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import random
from matplotlib import cm
import statsmodels.api as sm
from statsmodels.formula.api import ols as ols

In [12]:
df1 = pd.read_csv('SuperFilteredData.csv')

In [13]:
df1.head()

,StateAbbr,State,CityName,Measure,DataValueTypeID,Data_Value,PopulationCount,CA-latitude,CA-longitude,FL-latitude,FL-longitude,MA-latitude,MA-longitude
0,CA,California,Alameda,High blood pressure among adults aged >=18 Years,AgeAdjPrv,23.8,73812,38.429498,-122.793289,30.562583,-84.257936,42.117944,-72.615679
1,CA,California,Alameda,High blood pressure among adults aged >=18 Years,CrdPrv,25.8,73812,38.405848,-122.770651,30.537487,-84.208716,42.120109,-72.608316
2,CA,California,Alameda,High blood pressure among adults aged >=18 Years,CrdPrv,27.3,3646,38.470273,-122.766696,30.532277,-84.290747,42.109904,-72.604168
3,CA,California,Alameda,High blood pressure among adults aged >=18 Years,CrdPrv,25.3,4107,38.460995,-122.761087,30.524199,-84.229299,42.123747,-72.600152
4,CA,California,Alameda,High blood pressure among adults aged >=18 Years,CrdPrv,29.6,4896,38.443232,-122.758348,30.524117,-84.209533,42.103772,-72.594087


We do a shapiro test to check whether this ANOVA can be done are not.
Conditions to do 1 way ANOVA is :
First we do a shapiro test, if this test passes then we continue to do a One Way ANOVA, otherwise, we do a Levene's Test. 

The first value is the shapiro statistic, the second is the p-value

In [44]:
shapiro(df1[df1['CityName'] == 'Alameda'].Data_Value)

(0.9713255763053894, 0.8222182393074036)

In [38]:
shapiro(df1[df1['State'] == 'California'].Data_Value)

C:\Users\Kristin Synnott\Anaconda2\lib\site-packages\scipy\stats\morestats.py:1309: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.9439206123352051, 1.9291675958359757e-41)

In [36]:
shapiro(df1[df1['State'] == 'Florida'].Data_Value)

(0.9867967367172241, 8.829720843372968e-10)

In [35]:
shapiro(df1[df1['State'] == 'Massachusetts'].Data_Value)

(0.9778867363929749, 5.551264621317387e-07)

In [41]:
cali = pd.read_csv('california.csv')
dripdrop = cali.drop(['CA-latitude','CA-longitude','FL-latitude','FL-longitude','MA-latitude','MA-longitude','Measure','DataValueTypeID','PopulationCount'], axis = 1)
california_sample = dripdrop.sample(n=5000)
california_sample.head()

,StateAbbr,State,CityName,Data_Value
967,CA,California,Fremont,25.8
5042,CA,California,Stockton,36.2
116,CA,California,Anaheim,24.3
297,CA,California,Berkeley,24.2
4815,CA,California,Santa Clarita,23.5


In [42]:
shapiro(california_sample[california_sample['State'] == 'California'].Data_Value)

(0.9497498869895935, 2.4606582430983353e-38)

In [49]:
alpha = .05
p = 0

In [55]:
if p > alpha:
    print p, "Sample looks Gaussian aka fail to reject H0"#means that it passes, data is normally distributed
if p < alpha: 
    print p, "Sample does not look Gaussian, reject the H0"#means it fails

0 Sample does not look Gaussian, reject the H0


After writing the code for testing alpha, we ran the data through SPSS to check out work. 

I tried to do it by state, however, because the sample size is so large, you get errors, so I just did one city for now.

If the p-value is less than the significance level (in this case 0.05, a 95% confidence interval), the null hypothesis that the data is normally distributed can be rejected. In short, if the p-value is less than 0.05, we can assume the data is not normally distributed. 

In [56]:
#Levenes Test
#Since our Shapiro failed, we do a Levene's test
ldf1=df1[df1['State'] == 'Florida']
ldf2=df1[df1['State'] == 'Massachusetts']
ldf3=california_sample[california_sample['State'] == 'California']

In [35]:
x1=ldf1['Data_Value'].mean()
y1=ldf2['Data_Value'].mean()
z1=ldf3['Data_Value'].mean()

In [37]:
print(x1,y1,z1)

(32.17626745040412, 28.44372549019608, 26.870520000000052)


In [39]:
# Actual Levene's test

levene(ldf1['Data_Value'],ldf2['Data_Value'],ldf3['Data_Value'])

LeveneResult(statistic=214.80366940467528, pvalue=3.261335657269471e-91)

you want p value >.05 so homogenuity fails. Null is all variances are equal.

Since the states failed as far as normality, we decided to run the Shaprio Wilk for all 163 cities in SPSS to see if the cities were normal                                                      

This turned put to be normal, so we can use the cities in our analysis